In [135]:
# Should split this into a second file for regression task

# Standard libs
import numpy as np
import pandas as pd

# Debugging
from icecream import ic

# Predictive modelling
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.svm import SVR, SVC

# NN functions
from keras.layers import Dense, Dropout, BatchNormalization
from keras.models import Sequential

# Metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# CV ; splitting
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

We can actually try to predict every numeric and categorical feature in this dataset, but for now let's do classification on possum population class `Pop`.

In [2]:
d = pd.read_csv('possum.csv')

In [3]:
d.head()

,case,site,Pop,sex,age,hdlngth,skullw,totlngth,taill,footlgth,earconch,eye,chest,belly
0,1,1,Vic,m,8.0,94.1,60.4,89.0,36.0,74.5,54.5,15.2,28.0,36.0
1,2,1,Vic,f,6.0,92.5,57.6,91.5,36.5,72.5,51.2,16.0,28.5,33.0
2,3,1,Vic,f,6.0,94.0,60.0,95.5,39.0,75.4,51.9,15.5,30.0,34.0
3,4,1,Vic,f,6.0,93.2,57.1,92.0,38.0,76.1,52.2,15.2,28.0,34.0
4,5,1,Vic,f,2.0,91.5,56.3,85.5,36.0,71.0,53.2,15.1,28.5,33.0


In [4]:
d.isna().sum()

case        0
site        0
Pop         0
sex         0
age         2
hdlngth     0
skullw      0
totlngth    0
taill       0
footlgth    1
earconch    0
eye         0
chest       0
belly       0
dtype: int64

**Preprocessing**

In [5]:
d['age'] = d['age'].fillna(d['age'].median())
d['footlgth'] = d['footlgth'].fillna(d['footlgth'].mean())

In [6]:
d.isna().sum()

case        0
site        0
Pop         0
sex         0
age         0
hdlngth     0
skullw      0
totlngth    0
taill       0
footlgth    0
earconch    0
eye         0
chest       0
belly       0
dtype: int64

In [7]:
# One-hot encode categorical variables and concatenate with dataframe
d = pd.concat([pd.get_dummies(d['site'], drop_first=True), 
               pd.get_dummies(d['Pop'], drop_first=True),
               pd.get_dummies(d['sex'], drop_first=True),
              d.drop(['site','Pop','sex', 'case'], axis = 1)], axis = 1)

In [8]:
# Convert age to integer value for categorization
d = d.astype({'age': int})

In [9]:
# Create models
# For later...
# regressors = [LinearRegression(), 
#               DecisionTreeRegressor(), 
#               RandomForestRegressor(), 
#               BaggingRegressor(), 
#               AdaBoostRegressor(), 
#               GradientBoostingRegressor(), 
#               KNeighborsRegressor()]


# classifiers = [LogisticRegression(), 
#                DecisionTreeClassifier(), 
#                RandomForestClassifier(), 
#                BaggingClassifier(),
#                AdaBoostClassifier(), 
#                GradientBoostingClassifier(), 
#                KNeighborsClassifier()]

In [10]:
d.head()

,2,3,4,5,6,7,other,m,age,hdlngth,skullw,totlngth,taill,footlgth,earconch,eye,chest,belly
0,0,0,0,0,0,0,0,1,8,94.1,60.4,89.0,36.0,74.5,54.5,15.2,28.0,36.0
1,0,0,0,0,0,0,0,0,6,92.5,57.6,91.5,36.5,72.5,51.2,16.0,28.5,33.0
2,0,0,0,0,0,0,0,0,6,94.0,60.0,95.5,39.0,75.4,51.9,15.5,30.0,34.0
3,0,0,0,0,0,0,0,0,6,93.2,57.1,92.0,38.0,76.1,52.2,15.2,28.0,34.0
4,0,0,0,0,0,0,0,0,2,91.5,56.3,85.5,36.0,71.0,53.2,15.1,28.5,33.0


In [11]:
X = d.drop('other', axis = 1)
y = d['other']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.15, random_state=42)

# Logistic Regression Classification

In [12]:
# Predict where the population came from
lrc = LogisticRegression(max_iter=1000, random_state=42)
# Site has numeric dtype names and forces using .values attr
lrc.fit(X_train.values, y_train.values)

# Predict only first 3
ic(lrc.predict(X_test[:3].values))
# Accuracy of 1st 3
ic(accuracy_score(lrc.predict(X_test[:3].values), y_test[:3].values))
# Accuracy of entire test set
ic(accuracy_score(lrc.predict(X_test.values), y_test.values))
# Confusion matrix
ic(confusion_matrix(lrc.predict(X_test.values), y_test.values));
print(classification_report(lrc.predict(X_test.values), y_test.values))

ic| lrc.predict(X_test[:3].values): array([0, 1, 1], dtype=uint8)
ic| accuracy_score(lrc.predict(X_test[:3].values), y_test[:3].values): 1.0
ic| accuracy_score(lrc.predict(X_test.values), y_test.values): 1.0
ic| confusion_matrix(lrc.predict(X_test.values), y_test.values): array([[7, 0],
                                                                        [0, 9]], dtype=int64)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       1.00      1.00      1.00         9

    accuracy                           1.00        16
   macro avg       1.00      1.00      1.00        16
weighted avg       1.00      1.00      1.00        16



The Logistic Regression classifier's is predicting one (or none) possum to be part of the 'other' population when it is actually of the 'Victorian' population (False Positive in this case).

Note that precision and recall aren't of high importance as predicting possum population is not a critical assessment; accuracy is sufficient.

The test set size is quite small due to having only 104 possums for our sample. We can use cross validation to get a better understanding of how our model is performing.

In [13]:
cv = cross_val_score(LogisticRegression(max_iter=1000), 
                     X_train.values, 
                     y_train.values, cv = 5);
ic(cv)
cv = cross_val_score(LogisticRegression(max_iter=1000), X_train.values, y_train.values, cv = 15);
ic(cv);


ic| cv: array([0.94444444, 1.        , 1.        , 1.        , 1.        ])
ic| cv: array([1.        , 0.83333333, 1.        , 0.83333333, 1.        ,
               1.        , 1.        , 1.        , 1.        , 1.        ,
               1.        , 0.83333333, 1.        , 1.        , 1.        ])


The classifier is working pretty well. We should expect a poorer performance on at least one of the splits due to the sample size.

# Decision Tree Classification

In [14]:
dtc = DecisionTreeClassifier(random_state=42)
dtc.fit(X_train.values, y_train.values);

DecisionTreeClassifier(random_state=42)

In [15]:
# Just to show the unregularized tree will severly overfit the data
dtc.predict(X_test.values) == y_test.values

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [34]:
max_depth = 10 # Set up depth of heirarchy
min_samples_split = 10

dtc_params = {
    'criterion' : ['gini', 'entropy', 'log_loss'], # Expect similar trees with all methods
    'splitter' : ['random'],               # Random should speed up training if the task was bigger
    'max_depth' : list(range(1, max_depth)),          # Prevent tree from becoming "pure" (overfit)
    'min_samples_split' : list(range(1, min_samples_split)), # Regularizing : prevents unnecessary computations
    'min_samples_leaf' : [10]                      # Regularizing
}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid=dtc_params, verbose = 0)
gs.fit(X_train.values, y_train.values)
ic(accuracy_score(gs.predict(X_test.values), y_test.values));

ic| accuracy_score(gs.predict(X_test.values), y_test.values): 0.875


In [54]:
gs.best_params_

{'criterion': 'gini',
 'max_depth': 2,
 'min_samples_leaf': 10,
 'min_samples_split': 1,
 'splitter': 'random'}

In [134]:
# Random forest classifier dictionary additions
# "additions" as its parameters are similar those of Decision Tree



''' Want to add a chunk to add parameters to dtc_params...
cannot use d1.update(d2) as it works in-place AND makes a copy'''
# rfc_addtns = {'n_estimators' : list(range(20,110,10))}

# rfc_addtns

# dtc_params.update(rfc_addtns)

# del dtc_params['n_estimators']

# rfc_params = dtc_params.update(rfc_addtns)

# dtc_params

# gs.best_params_

' Want to add a chunk to add parameters to dtc_params...\ncannot use d1.update(d2) as it works in-place AND makes a copy'

The accuracy is 100%, but on multiple runs we see a drop... < 10%. Again not surprising.

In [36]:
# store importances because the attr name is long
importances = gs.best_estimator_.feature_importances_.round(2)
col_names = X_train.columns # also longish

# zip features with its importance
feats = list(zip(importances, col_names))
ic(feats)

# sort by importance
feats.sort(key=lambda feat: feat[0])
# important features to the front
feats.reverse()

ic| feats: [(0.0, 2),
            (0.0, 3),
            (0.0, 4),
            (0.0, 5),
            (0.0, 6),
            (0.0, 7),
            (0.0, 'm'),
            (0.0, 'age'),
            (0.0, 'hdlngth'),
            (0.0, 'skullw'),
            (0.0, 'totlngth'),
            (0.19, 'taill'),
            (0.0, 'footlgth'),
            (0.81, 'earconch'),
            (0.0, 'eye'),
            (0.0, 'chest'),
            (0.0, 'belly')]


In [37]:
# Print important features
headers = ['feature', 'importance']
print('{:>15} {:>15}'.format(headers[0], headers[1]))
print('---------------------------------------------')

for (i, j) in feats:
    
    # If i is not 0
    if i:
        print(f'{j:>15}', f'{i:>15}')
        
    # At the first 0 occurence, break
    # bc everything else is 0
    else: 
        break

        feature      importance
---------------------------------------------
       earconch            0.81
          taill            0.19


What is moderately surprising is that the decision tree finds the possum's ear size as the best predictor of the possum's population for the decision tree. This is confirmed by choosing "best" or "random" for the `splitter` argument.

# Ensemble: Random Forest Classification

Since our task in predicting possum population is not too difficult, and we are only going to look at stronger algorithms, we will reduce the training-test split to 50/50 and see how the algorithms perform. We will also quickly revisit the Logistic Regression and Decision Tree classifiers.

In [118]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5, random_state=42)

In [129]:
############## LOGISTIC REGRESSION #################

lrc_cv5 = cross_val_score(LogisticRegression(max_iter=1000), 
                         X.values, 
                         y.values, cv = 5);
ic(lrc_cv5)

lrc_cv10 = cross_val_score(LogisticRegression(max_iter=1000), 
                         X.values, 
                         y.values, cv = 10);
ic(lrc_cv10)

############## DECISION TREES ####################

dtc_cv5 = cross_val_score(DecisionTreeClassifier(**gs.best_params_,random_state=42),
                         X.values, 
                         y.values, cv = 5);

ic(dtc_cv5);

dtc_cv10 = cross_val_score(DecisionTreeClassifier(**gs.best_params_,random_state=42),
                         X.values, 
                         y.values, cv = 10);

ic(dtc_cv10);

ic| lrc_cv5: array([1.        , 1.        , 0.95238095, 0.95238095, 0.95      ])
ic| lrc_cv10: array([1. , 1. , 1. , 1. , 0.9, 1. , 0.9, 1. , 0.9, 1. ])
ic| dtc_cv5: array([0.66666667, 0.80952381, 0.95238095, 0.95238095, 0.7       ])
ic| dtc_cv10: array([0.90909091, 1.        , 0.72727273, 0.90909091, 0.9       ,
                     0.8       , 1.        , 0.9       , 0.7       , 0.8       ])


The Logistic Regression classifier is a still a strong baseline classifier, but notice now that the performance of the Decision Tree degrades when trained on less samples, even when the classifier has the best parameters from the earlier `gridsearchCV`. Overfitting has become apparent for the Decision Tree and it's likely that performance will degrade further with more observations.

Below we see that the Random Forest is a strong classifier and will perform well on both small splits of data and large.

In [120]:
rfc_params = {
    'n_estimators' : 100,
    'max_depth' : 10
}

rfc = RandomForestClassifier(**rfc_params, random_state=42)

In [121]:
rfc.fit(X_train.values, y_train.values)

RandomForestClassifier(max_depth=10, random_state=42)

In [127]:
rfc.predict_proba(X_test.values).round(2)[:10]

array([[0.68, 0.32],
       [0.04, 0.96],
       [0.1 , 0.9 ],
       [0.17, 0.83],
       [0.95, 0.05],
       [0.09, 0.91],
       [0.02, 0.98],
       [0.15, 0.85],
       [0.91, 0.09],
       [0.83, 0.17]])

In [123]:
rfc.predict(X_test.values).round(2)

array([0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 0], dtype=uint8)

In [124]:
accuracy_score(rfc.predict(X_test.values), y_test.values)

0.9807692307692307

In [133]:
########### RFC ON 50/50 TRAINING SET ##################

rfc_5050 = cross_val_score(RandomForestClassifier(**rfc_params, random_state=42),
               X = X_train.values, y = y_train.values, cv = 10)

ic(rfc_5050)
########### RFC ON FULL SET ##################
rfc_full = cross_val_score(RandomForestClassifier(**rfc_params, random_state=42),
               X = X.values, y = y.values, cv = 10)

ic(rfc_full);

ic| rfc_5050: array([1. , 1. , 1. , 1. , 1. , 1. , 0.8, 1. , 1. , 1. ])
ic| rfc_full: array([1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 0.9, 1. ])


array([1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 0.9, 1. ])

# Ensemble: Bagging Classifier
We will also look at the Bagging Classifier. Perhaps I should have done this first considering Random Forests are Bagging Classifiers that add random subsetting of features.

In [ ]:
BaggingClassifier()

In [19]:
# Function for testing

'''
1. Input kwargs ()
2. Do cross validation (cv)
3. write outputs (verbose)
4. df (T/F)
5. save

'''

'\n1. Input kwargs ()\n2. Do cross validation (cv)\n3. write outputs (verbose)\n4. df (T/F)\n5. save\n\n'